In [ ]:
!pip install pyspark
import pyspark
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 9.4 MB/s eta 0:00:00


In [ ]:
#Import Libraries

from pyspark.sql import SparkSession

# Create a new Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

from pyspark.sql.functions import col, count, when, round, mean, to_date, concat, lit
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum as _sum
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import types as T
import datetime
from pyspark.sql.functions import udf, col
from pyspark.sql.types import TimestampType, StringType
import datetime

In [ ]:
import pandas as pd

In [ ]:
#Read as pandas df

airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

In [ ]:
#convert to spark dataframe
flights_df = spark.read.csv("flights.csv", header=True, inferSchema=True)
airlines_df = spark.read.csv("airlines.csv", header=True, inferSchema=True)
airports_df = spark.read.csv("airports.csv", header=True, inferSchema=True)


In [ ]:
flights_df.show(5)


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [ ]:
#rename columns

airlines_df = airlines_df.withColumnRenamed("AIRLINE", "FLIGHT")

In [ ]:
airports_df.show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



In [ ]:
airlines_df.show(5)

+---------+--------------------+
|IATA_CODE|              FLIGHT|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
+---------+--------------------+
only showing top 5 rows



In [ ]:
# Join Flights with Airlines to get airline names using inner join, using columns AIRLINE and IATA_code
flights_airlines_df = flights_df.join(airlines_df, flights_df.AIRLINE == airlines_df.IATA_CODE, "inner") \
                                .drop("IATA_CODE")  # Drop duplicate IATA_CODE column
flights_airlines_df_sorted = flights_airlines_df.orderBy("AIRLINE")
flights_airlines_df_sorted.show(500)

flights_airlines_df_outer = flights_df.join(airlines_df, flights_df.AIRLINE == airlines_df.IATA_CODE, "outer") \
                                .drop("IATA_CODE")  # Drop duplicate IATA_CODE column

flights_airlines_df_outer_sorted = flights_airlines_df.orderBy("AIRLINE")
flights_airlines_df_outer_sorted.show(500)


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|              FLIGHT|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+-------

In [ ]:
# Join for origin airport information
#join the resulting DataFrame (flights_airlines_df) with the Airports DataFrame using the ORIGIN_AIRPORT column from the Flights DataFrame and the IATA_CODE from the Airports DataFrame.
flights_airlines_origin_df = flights_airlines_df.join(airports_df, flights_airlines_df.ORIGIN_AIRPORT == airports_df.IATA_CODE, "inner") \
                                                .drop("IATA_CODE") \
                                                .withColumnRenamed("AIRPORT", "ORIGIN_AIRPORT_NAME") \
                                                .withColumnRenamed("CITY", "ORIGIN_CITY") \
                                                .withColumnRenamed("STATE", "ORIGIN_STATE") \
                                                .withColumnRenamed("COUNTRY", "ORIGIN_COUNTRY") \
                                                .withColumnRenamed("LATITUDE", "ORIGIN_LATITUDE") \
                                                .withColumnRenamed("LONGITUDE", "ORIGIN_LONGITUDE")


In [ ]:
# Join for destination airport information
#join the DataFrame (flights_airlines_origin_df) again with the Airports DataFrame, but this time we use the DESTINATION_AIRPORT column from the Flights DataFrame and the IATA_CODE from the Airports DataFrame.
flights_final_df = flights_airlines_origin_df.join(airports_df, flights_airlines_origin_df.DESTINATION_AIRPORT == airports_df.IATA_CODE, "inner") \
                                             .drop("IATA_CODE") \
                                             .withColumnRenamed("AIRPORT", "DESTINATION_AIRPORT_NAME") \
                                             .withColumnRenamed("CITY", "DESTINATION_CITY") \
                                             .withColumnRenamed("STATE", "DESTINATION_STATE") \
                                             .withColumnRenamed("COUNTRY", "DESTINATION_COUNTRY") \
                                             .withColumnRenamed("LATITUDE", "DESTINATION_LATITUDE") \
                                             .withColumnRenamed("LONGITUDE", "DESTINATION_LONGITUDE")\
                                             .withColumnRenamed("AIRLINE", "AIRL")\
                                             .withColumnRenamed("AIRLINE", "AIRLINE")


In [ ]:
flights_final_df.show(5)

+----+-----+---+-----------+----+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+------------+--------------+---------------+----------------+------------------------+----------------+-----------------+-------------------+--------------------+---------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRL|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AI

In [ ]:
from pyspark.sql import functions as F

#Create new date column
flights_final_df = flights_final_df.withColumn(
    "DATE",
    F.to_date(F.concat(
        F.lpad(F.col("YEAR"), 4, '0'),
        F.lit("-"),
        F.lpad(F.col("MONTH"), 2, '0'),
        F.lit("-"),
        F.lpad(F.col("DAY"), 2, '0')
    ), "yyyy-MM-dd")
)

#DataFrame with the new DATE column
flights_final_df.show()


+----+-----+---+-----------+----+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+--------------------+--------------+------------+--------------+---------------+----------------+------------------------+-----------------+-----------------+-------------------+--------------------+---------------------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRL|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_

In [ ]:
"""# Check for invalid time entries in your time-related columns
# Replace 'YOUR_TIME_COLUMN' with the actual time column names you have
invalid_times_df = flights_final_df.filter((col('DEPARTURE_TIME') >= 2400) | (col('DEPARTURE_TIME') < 0))

# Show the invalid time entries if any
invalid_times_df.show()"""


"# Check for invalid time entries in your time-related columns\n# Replace 'YOUR_TIME_COLUMN' with the actual time column names you have\ninvalid_times_df = flights_final_df.filter((col('DEPARTURE_TIME') >= 2400) | (col('DEPARTURE_TIME') < 0))\n\n# Show the invalid time entries if any\ninvalid_times_df.show()"

In [ ]:
flights_final_df.dropna()




DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRL: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int, FLIGHT: string, ORIGIN_AIRPORT_NAME: string, ORIGIN_CITY: string, ORIGIN_STATE: string, ORIGIN_COUNTRY: string, ORIGIN_LATITUDE: double, ORIGIN_LONGITUDE: double, DESTINATION_AIRPORT_NAME: string, DESTINATION_CITY: string, DESTINATION_STATE: string, DESTINATION_COUNTRY: string, DESTINATION_LATITUDE: double, DESTINATION_LONGITUDE: double, DATE: date]

In [ ]:
flights_final_df.show(5)


+----+-----+---+-----------+----+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+------------+--------------+---------------+----------------+------------------------+----------------+-----------------+-------------------+--------------------+---------------------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRL|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DE

In [ ]:
columns_to_fill = ['DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
                   'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

# Step 1: Calculate mean for each specified column
mean_values = {c: flights_final_df.select(mean(c)).first()[0] for c in columns_to_fill if flights_final_df.select(c).filter(flights_final_df[c].isNull()).count() > 0}

# Step 2: Replace nulls in these columns with the calculated means
final_df = flights_final_df.fillna(mean_values)

# Show the updated DataFrame

final_df_sorted = final_df.orderBy("DATE")
final_df_sorted.show()

+----+-----+---+-----------+----+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+--------------------+--------------+------------+--------------+---------------+----------------+------------------------+-----------------+-----------------+-------------------+--------------------+---------------------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRL|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_

In [ ]:
from pyspark.sql.functions import udf, col, concat_ws, lit
from pyspark.sql.types import StringType, TimestampType
import datetime

# UDF to format time in HHMM to datetime.time
def format_time(time_value):
    if time_value is None:
        return None
    time_value = int(time_value)
    if time_value == 2400:
        return datetime.time(0, 0)
    time_str = "{:04d}".format(time_value)
    return datetime.time(int(time_str[:2]), int(time_str[2:4]))

format_time_udf = udf(format_time, StringType())

# UDF to combine date and time into a datetime.datetime object
def combine_date_time(date, time_str):
    if date is None or time_str is None:
        return None
    time_obj = datetime.datetime.strptime(time_str, "%H:%M").time()
    return datetime.datetime.combine(date, time_obj)

combine_date_time_udf = udf(combine_date_time, TimestampType())

# Convert `DEPARTURE_TIME` and `SCHEDULED_DEPARTURE`
final_df = flights_final_df.withColumn("FORMATTED_DEPARTURE_TIME", format_time_udf(col("DEPARTURE_TIME"))) \
                           .withColumn("FORMATTED_SCHEDULED_DEPARTURE", format_time_udf(col("SCHEDULED_DEPARTURE")))

# Combine `DATE` and `FORMATTED_DEPARTURE_TIME` into `FLIGHT_DATETIME`
final_df = final_df.withColumn("FLIGHT_DATETIME", combine_date_time_udf(col("DATE"), col("FORMATTED_DEPARTURE_TIME")))

final_df.select("DATE", "DEPARTURE_TIME", "FORMATTED_DEPARTURE_TIME", "FLIGHT_DATETIME").show(5)


In [ ]:
final_df.coalesce(1).write.mode("overwrite").csv('output_file_path.csv', header=True)

In [ ]:
"""from pyspark.sql.functions import when

# Update the YEAR column from 2015 to 2023
final_df = final_df.withColumn("YEAR", when(final_df["YEAR"] == 2015, 2023).otherwise(final_df["YEAR"]))

# Show the updated DataFrame
final_df.show()"""


'from pyspark.sql.functions import when\n\n# Update the YEAR column from 2015 to 2023\nfinal_df = final_df.withColumn("YEAR", when(final_df["YEAR"] == 2015, 2023).otherwise(final_df["YEAR"]))\n\n# Show the updated DataFrame\nfinal_df.show()'

In [ ]:
"""final_df.write.option("delimiter", ",") \
         .option("quote", "\"") \
         .mode("overwrite") \
         .csv('output_file_path.csv', header=True)"""

In [ ]:
"""final_df.coalesce(1).write.mode("overwrite").csv('output_file_path.csv', header=True)
"""